# Time Decay
**Truth Premise: When a day ends and a new day begins in life of options premium, its value depreciates because of time decay because it gets closer to expiry and the probability of its contract going as expected by the buyer gets lesser.**

## Strategy
1. Short both put and call at day closing right before the expiry
2. Square of in the morning

## Pseudocode
```python
nifty_at_0320
at_the_money_call_strike
at_the_money_put_strike
call_strike_premium_at_0320
put_strike_premium_at_0320
call_strike_premium_at_0915
put_strike_premium_at_0915
call_strike_premium_at_0916
put_strike_premium_at_0916
call_strike_premium_at_0917
put_strike_premium_at_0917
call_strike_premium_at_0918
put_strike_premium_at_0918
pnl_0915
pnl_0916
pnl_0917
pnl_0918
```

In [1]:
import datetime as dt
import utils as ut
import pandas as pd
import icharts as ic
from functools import cache
from constants import *


TEST_START = dt.datetime.strptime("2021-01-01", "%Y-%m-%d")
TEST_END = dt.datetime.strptime("2023-12-31", "%Y-%m-%d")
SYMBOL = "NIFTY 50"
IC_SYMBOL = "NIFTY"
INTERVAL = ut.INTERVAL_MIN1
EXCHANGE = ut.EXCHANGE_NSE
pickle_file_name = "ocdf_2024_02_17.pkl"
# pickle_file_name = "test_analyzer_ocdf_2024_02_17.pkl"

pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", 200)
# pd.set_option('precision', 2)
pd.set_option("display.precision", 2)
# pd.set_option('display.float_format', lambda x: '%.2f' % x)

def build_date_range(date_start, date_end, symbol):
    date_range = []
    cur_date = date_start
    while cur_date < date_end:
        if cur_date.weekday() not in [5, 6]:
            has_data, _ = ut.has_data(symbol, cur_date, interval=INTERVAL, exchange=EXCHANGE)
            if has_data:
                date_range.append(cur_date)
        cur_date += dt.timedelta(days=1)
    return date_range

all_dates = pd.DataFrame({"trade_date": build_date_range(TEST_START, TEST_END, SYMBOL)})
all_dates_shuffled = all_dates.sample(frac=1, random_state=42)

train_size = int(0.5 * len(all_dates_shuffled))
train_dates = all_dates_shuffled.iloc[:train_size]
test_dates = all_dates_shuffled.iloc[train_size:]
train_dates = train_dates.sort_values(by="trade_date")
train_dates.set_index("trade_date", inplace=True)
test_dates = test_dates.sort_values(by="trade_date")
test_dates.set_index("trade_date", inplace=True)
# train_dates = test_dates

@cache
def get_intraday_data(date):
    return ut.get_data(symbol=SYMBOL, date=date, interval=INTERVAL, exchange=EXCHANGE)

@cache
def get_symbol_first_candle(symbol, trade_date):
    data = ut.get_data(symbol=SYMBOL, date=trade_date, interval=INTERVAL, exchange=EXCHANGE)
    return data.iloc[0].open, data.iloc[0].high, data.iloc[0].low, data.iloc[0].close

@cache
def get_first_candle_close(symbol, trade_date):
    data = ut.get_data(symbol=SYMBOL, date=trade_date, interval=INTERVAL, exchange=EXCHANGE)
    return data.iloc[0].close

@cache
def get_last_trading_day(date):
    return ut.get_last_trading_day(SYMBOL, date, interval=INTERVAL, exchange=ut.EXCHANGE_NSE)

train_dates["previous_trading_day"] = None
train_dates["previous_trading_day"] = train_dates.apply(lambda row: get_last_trading_day(row.name), axis=1)
train_dates["expiry"] = train_dates.apply(lambda row: ut.find_nclosest_expiry(SYMBOL, row.name, 3), axis=1)
# train_dates = train_dates.loc[train_dates.expiry == train_dates.index + pd.Timedelta(days=1)]

day_before_expiry = 1

def get_nifty_price(trade):
    data = ut.get_data(symbol=SYMBOL, date=trade.previous_trading_day.date(), interval=INTERVAL, exchange=EXCHANGE)
    t = dt.time(hour=15, minute=20)
    try:
        return data.loc[data.index.time == t].iloc[0].open
    except IndexError:
        return pd.NA
    except AttributeError as e:
        return pd.NA

train_dates = train_dates.copy()
# train_dates["trade_day_before_expiry"] = train_dates.expiry - pd.Timedelta(days=day_before_expiry)
train_dates.loc[:, "nifty_at_0320"] = train_dates.apply(get_nifty_price, axis=1)
train_dates = train_dates.loc[train_dates.nifty_at_0320.notna()]
train_dates["atm_strike"] = train_dates.apply(lambda trade: round(trade.nifty_at_0320 / 50) * 50, axis=1)
# train_dates["call_atm_strike"] = train_dates["atm_strike"] - 100
# train_dates["put_atm_strike"] = train_dates["atm_strike"] + 300



In [2]:
def get_premium_df(trade, strike_price, td, option_type, tm):
    pr = ic.get_opt_pre_df(symbol=SYMBOL, expiry=trade.expiry, cur_dt=td, strike_price=strike_price, option_type=option_type)
    if pr.shape[0] == 0:
        return pd.NA
    # x = pr.loc[(pr.index.date == td) & (pr.index.time >= tm)]
    # # print(x.iloc[0])
    # if x.shape[0] == 0:
    #     print(tm)
    #     print(td)
    #     print(pr.loc[(pr.index.date == td)])
    #     print(trade)
    return pr.loc[(pr.index.date == td) & (pr.index.time >= tm)].iloc[0].close

train_dates["call_at_0320"] = train_dates.apply(lambda trade: get_premium_df(trade, trade.call_atm_strike, trade.name.date(), OPTION_TYPE_CALL, dt.time(hour=15, minute=20)), axis=1)
train_dates["put_at_0320"] = train_dates.apply(lambda trade: get_premium_df(trade, trade.put_atm_strike, trade.name.date(), OPTION_TYPE_PUT, dt.time(hour=15, minute=20)), axis=1)
# train_dates["call_at_0915"] = train_dates.apply(lambda trade: get_premium_df(trade, trade.call_atm_strike, trade.expiry.date(), OPTION_TYPE_CALL, dt.time(hour=9, minute=15)), axis=1)
train_dates["put_at_0915"] = train_dates.apply(lambda trade: get_premium_df(trade, trade.put_atm_strike, trade.expiry.date(), OPTION_TYPE_PUT, dt.time(hour=9, minute=15)), axis=1)
# train_dates["call_at_0916"] = train_dates.apply(lambda trade: get_premium_df(trade, trade.call_atm_strike, trade.expiry.date(), OPTION_TYPE_CALL, dt.time(hour=9, minute=16)), axis=1)
train_dates["put_at_0916"] = train_dates.apply(lambda trade: get_premium_df(trade, trade.put_atm_strike, trade.expiry.date(), OPTION_TYPE_PUT, dt.time(hour=9, minute=16)), axis=1)
# train_dates["call_at_0917"] = train_dates.apply(lambda trade: get_premium_df(trade, trade.call_atm_strike, trade.expiry.date(), OPTION_TYPE_CALL, dt.time(hour=9, minute=17)), axis=1)
train_dates["put_at_0917"] = train_dates.apply(lambda trade: get_premium_df(trade, trade.put_atm_strike, trade.expiry.date(), OPTION_TYPE_PUT, dt.time(hour=9, minute=17)), axis=1)
# train_dates["call_at_0918"] = train_dates.apply(lambda trade: get_premium_df(trade, trade.call_atm_strike, trade.expiry.date(), OPTION_TYPE_CALL, dt.time(hour=9, minute=18)), axis=1)
train_dates["put_at_0918"] = train_dates.apply(lambda trade: get_premium_df(trade, trade.put_atm_strike, trade.expiry.date(), OPTION_TYPE_PUT, dt.time(hour=9, minute=18)), axis=1)
train_dates["put_at_10"] = train_dates.apply(lambda trade: get_premium_df(trade, trade.put_atm_strike, trade.expiry.date(), OPTION_TYPE_PUT, dt.time(hour=10, minute=0)), axis=1)
train_dates["put_at_11"] = train_dates.apply(lambda trade: get_premium_df(trade, trade.put_atm_strike, trade.expiry.date(), OPTION_TYPE_PUT, dt.time(hour=11, minute=0)), axis=1)
# train_dates["put_at_12"] = train_dates.apply(lambda trade: get_premium_df(trade, trade.put_atm_strike, trade.expiry.date(), OPTION_TYPE_PUT, dt.time(hour=12, minute=0)), axis=1)
# train_dates["put_at_13"] = train_dates.apply(lambda trade: get_premium_df(trade, trade.put_atm_strike, trade.expiry.date(), OPTION_TYPE_PUT, dt.time(hour=13, minute=0)), axis=1)
# train_dates["put_at_14"] = train_dates.apply(lambda trade: get_premium_df(trade, trade.put_atm_strike, trade.expiry.date(), OPTION_TYPE_PUT, dt.time(hour=14, minute=0)), axis=1)
# train_dates["put_at_15"] = train_dates.apply(lambda trade: get_premium_df(trade, trade.put_atm_strike, trade.expiry.date(), OPTION_TYPE_PUT, dt.time(hour=15, minute=0)), axis=1)
# train_dates["put_at_1520"] = train_dates.apply(lambda trade: get_premium_df(trade, trade.put_atm_strike, trade.expiry.date(), OPTION_TYPE_PUT, dt.time(hour=15, minute=20)), axis=1)

AttributeError: 'Series' object has no attribute 'put_atm_strike'

In [50]:
# train_dates["call_pnl_0915"] = train_dates["call_at_0320"] - train_dates["call_at_0915"]
# train_dates["call_pnl_0916"] = train_dates["call_at_0320"] - train_dates["call_at_0916"]
# train_dates["call_pnl_0917"] = train_dates["call_at_0320"] - train_dates["call_at_0917"]
# train_dates["call_pnl_0918"] = train_dates["call_at_0320"] - train_dates["call_at_0918"]

train_dates["put_pnl_0915"] = train_dates["put_at_0320"] - train_dates["put_at_0915"]
train_dates["put_pnl_0916"] = train_dates["put_at_0320"] - train_dates["put_at_0916"]
train_dates["put_pnl_0917"] = train_dates["put_at_0320"] - train_dates["put_at_0917"]
train_dates["put_pnl_0918"] = train_dates["put_at_0320"] - train_dates["put_at_0918"]

# train_dates["pnl_0915"] = train_dates["call_pnl_0915"] + train_dates["put_pnl_0915"]
# train_dates["pnl_0916"] = train_dates["call_pnl_0916"] + train_dates["put_pnl_0916"]
# train_dates["pnl_0917"] = train_dates["call_pnl_0917"] + train_dates["put_pnl_0917"]
# train_dates["pnl_0918"] = train_dates["call_pnl_0918"] + train_dates["put_pnl_0918"]

train_dates["pnl_put_at_10"] = train_dates["put_at_0320"] - train_dates["put_at_10"]
train_dates["pnl_put_at_11"] = train_dates["put_at_0320"] - train_dates["put_at_11"]
# train_dates["pnl_put_at_12"] = train_dates["put_at_0320"] - train_dates["put_at_12"]
# train_dates["pnl_put_at_13"] = train_dates["put_at_0320"] - train_dates["put_at_13"]
# train_dates["pnl_put_at_14"] = train_dates["put_at_0320"] - train_dates["put_at_14"]
# train_dates["pnl_put_at_15"] = train_dates["put_at_0320"] - train_dates["put_at_15"]
# train_dates["pnl_put_at_1520"] = train_dates["put_at_0320"] - train_dates["put_at_1520"]


# print(f"PnL 9:15: {train_dates.pnl_0915.sum()}")
# print(f"PnL 9:16: {train_dates.pnl_0916.sum()}")
# print(f"PnL 9:17: {train_dates.pnl_0917.sum()}")
# print(f"PnL 9:18: {train_dates.pnl_0918.sum()}")
# print(f"AVG PnL 9:15: {train_dates.pnl_0915.sum()/train_dates.shape[0]}")

# print(f"Call PnL 9:15: {train_dates.call_pnl_0915.sum()}")
# print(f"Call PnL 9:16: {train_dates.call_pnl_0916.sum()}")
# print(f"Call PnL 9:17: {train_dates.call_pnl_0917.sum()}")
# print(f"Call PnL 9:18: {train_dates.call_pnl_0918.sum()}")
# print(f"AVG Call PnL 9:15: {train_dates.call_pnl_0915.sum()/train_dates.shape[0]}")

print(f"Put PnL 9:15: {train_dates.put_pnl_0915.sum()}")
print(f"Put PnL 9:16: {train_dates.put_pnl_0916.sum()}")
print(f"Put PnL 9:17: {train_dates.put_pnl_0917.sum()}")
print(f"Put PnL 9:18: {train_dates.put_pnl_0918.sum()}")
print(f"AVG Put PnL 9:15: {train_dates.put_pnl_0915.sum()/train_dates.shape[0]}")


print(f"Put pnl_put_at_10: {train_dates.pnl_put_at_10.sum()}")
print(f"Put pnl_put_at_11: {train_dates.pnl_put_at_11.sum()}")
# print(f"Put pnl_put_at_12: {train_dates.pnl_put_at_12.sum()}")
# print(f"Put pnl_put_at_13: {train_dates.pnl_put_at_13.sum()}")
# print(f"Put pnl_put_at_14: {train_dates.pnl_put_at_14.sum()}")
# print(f"Put pnl_put_at_15: {train_dates.pnl_put_at_15.sum()}")
# print(f"Put pnl_put_at_1520: {train_dates.pnl_put_at_1520.sum()}")
print(f"Shape: {train_dates.shape[0]}")

Put PnL 9:15: 1097.6
Put PnL 9:16: 1070.95
Put PnL 9:17: 1172.9499999999998
Put PnL 9:18: 1307.8000000000002
AVG Put PnL 9:15: 13.719999999999999
Put pnl_put_at_10: 1391.3499999999997
Put pnl_put_at_11: 1169.5999999999997
Shape: 80


In [3]:
def get_premium_df(trade, strike_price, td, option_type, tm):
    pr = ic.get_opt_pre_df(symbol=IC_SYMBOL, expiry=trade.expiry, cur_dt=td, strike_price=strike_price, option_type=option_type)
    if type(pr) == type(pd.NA) or pr.shape[0] == 0:
        return pd.NA
    x = pr.loc[(pr.index.date == td) & (pr.index.time >= tm)]
    # # print(x.iloc[0])
    if x.shape[0] == 0:
        # print(tm)
        # print(td)
        # print(pr.loc[(pr.index.date == td)])
        # print(trade)
        return pd.NA
    return x.iloc[0].close

# for i in range(-500, 750, 50):
#     train_dates["cur_put_atm_strike"] = train_dates["atm_strike"] + i
#     key1 = f"put_at_0918_{i}"
#     key2 = f"put_at_0320_{i}"
#     train_dates[key2] = train_dates.apply(lambda trade: get_premium_df(trade, trade.cur_put_atm_strike, trade.trade_day_before_expiry.date(), OPTION_TYPE_PUT, dt.time(hour=15, minute=20)), axis=1)
#     train_dates[key1] = train_dates.loc[train_dates[key2].notna()].apply(lambda trade: get_premium_df(trade, trade.cur_put_atm_strike, trade.trade_day_before_expiry.date() + dt.timedelta(days=1), OPTION_TYPE_PUT, dt.time(hour=9, minute=18)), axis=1)
#     pnl_key = f"put_pnl_0918_{i}"
#     train_dates[pnl_key] = train_dates.loc[train_dates[key2].notna()][key2] - train_dates.loc[train_dates[key2].notna()][key1]
#     print(f"Put PnL at 9:18 for strike: {i}, Total: {train_dates[pnl_key].sum()}, Per Day: {train_dates[pnl_key].mean()}, Days: {train_dates.loc[train_dates[pnl_key].notna()].shape[0]}")

# for i in range(0, -6000, -1000):
#     train_dates["cur_call_atm_strike"] = train_dates["atm_strike"] - i
#     key2 = f"call_at_0320_{i}"
#     key1 = f"call_at_0918_{i}"
#     train_dates[key2] = train_dates.apply(lambda trade: get_premium_df(trade, trade.cur_call_atm_strike, trade.trade_day_before_expiry.date(), OPTION_TYPE_CALL, dt.time(hour=15, minute=20)), axis=1)
#     train_dates[key1] = train_dates.loc[train_dates[key2].notna()].apply(lambda trade: get_premium_df(trade, trade.cur_call_atm_strike, trade.trade_day_before_expiry.date() + dt.timedelta(days=1), OPTION_TYPE_CALL, dt.time(hour=9, minute=18)), axis=1)
#     pnl_key = f"call_pnl_0918_{i}"
#     train_dates[pnl_key] = train_dates.loc[train_dates[key2].notna()][key2] - train_dates.loc[train_dates[key2].notna()][key1]
#     print(f"Call PnL at 9:18 for strike: {i}, Total: {train_dates[pnl_key].sum()}, Per Day: {train_dates[pnl_key].mean()}, Days: {train_dates.loc[train_dates[pnl_key].notna()].shape[0]}")

def get_next_1000(strike, i):
    divider = 500
    reminder = strike % divider
    if reminder != 0:
        return divider * (strike // divider) - i * divider
    return 1000 * (strike // divider) - (i+1) * divider

last_strike = None
for i in range(6):
    train_dates["cur_call_atm_strike"] = train_dates["atm_strike"].apply(lambda r: get_next_1000(r, i))
    key2 = f"call_at_0320_{i}"
    key1 = f"call_at_0918_{i}"
    train_dates[key2] = train_dates.apply(lambda trade: get_premium_df(trade, trade.cur_call_atm_strike, trade.previous_trading_day.date(), OPTION_TYPE_CALL, dt.time(hour=15, minute=29)), axis=1)
    train_dates[key1] = train_dates.loc[train_dates[key2].notna()].apply(lambda trade: get_premium_df(trade, trade.cur_call_atm_strike, trade.name.date(), OPTION_TYPE_CALL, dt.time(hour=9, minute=16)), axis=1)
    pnl_key = f"call_pnl_0918_{i}"
    train_dates[pnl_key] = train_dates.loc[train_dates[key2].notna()][key1] - train_dates.loc[train_dates[key2].notna()][key2]
    print(f"Call PnL at 9:18 for strike: {i}, Total: {train_dates[pnl_key].sum()}, Per Day: {train_dates[pnl_key].mean()}, Days: {train_dates.loc[train_dates[pnl_key].notna()].shape[0]}")


Call PnL at 9:18 for strike: 0, Total: 2324.150000000001, Per Day: 6.678591954022991, Days: 348
Call PnL at 9:18 for strike: 1, Total: 1084.2000000000007, Per Day: 9.346551724137937, Days: 116
Call PnL at 9:18 for strike: 2, Total: 492.4500000000003, Per Day: 12.959210526315797, Days: 38
Call PnL at 9:18 for strike: 3, Total: 324.44999999999936, Per Day: 27.037499999999948, Days: 12


In [5]:
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", 2000)

train_dates

,previous_trading_day,expiry,nifty_at_0320,atm_strike,cur_call_atm_strike,call_at_0320_0,call_at_0918_0,call_pnl_0918_0,call_at_0320_1,call_at_0918_1,call_pnl_0918_1,call_at_0320_2,call_at_0918_2,call_pnl_0918_2,call_at_0320_3,call_at_0918_3,call_pnl_0918_3
trade_date,,,,,,,,,,,,,,,,,
2021-01-01,2020-12-31,2021-01-07,13979.9,14000,10000,980.1,1006.15,-26.05,<NA>,NaN,NaN,<NA>,NaN,NaN,<NA>,NaN,NaN
2021-01-05,2021-01-04,2021-01-07,14129.55,14150,11000,182.55,139.35,43.2,<NA>,NaN,NaN,<NA>,NaN,NaN,<NA>,NaN,NaN
2021-01-11,2021-01-08,2021-01-14,14352.45,14350,11000,379.95,407.60,-27.65,<NA>,NaN,NaN,<NA>,NaN,NaN,<NA>,NaN,NaN
2021-01-12,2021-01-11,2021-01-14,14483.75,14500,11000,484.0,462.20,21.8,<NA>,NaN,NaN,<NA>,NaN,NaN,<NA>,NaN,NaN
2021-01-14,2021-01-13,2021-01-14,14566.6,14550,11000,563.25,551.85,11.4,<NA>,NaN,NaN,<NA>,NaN,NaN,<NA>,NaN,NaN
2021-01-15,2021-01-14,2021-01-21,14593.15,14600,11000,634.7,626.45,8.25,<NA>,NaN,NaN,<NA>,NaN,NaN,<NA>,NaN,NaN
2021-01-18,2021-01-15,2021-01-21,14441.85,14450,11000,486.0,429.40,56.6,<NA>,NaN,NaN,<NA>,NaN,NaN,<NA>,NaN,NaN
2021-01-22,2021-01-21,2021-01-28,14621.6,14600,11000,666.0,585.00,81.0,<NA>,NaN,NaN,<NA>,NaN,NaN,3635.0,3557.95,77.05
2021-01-28,2021-01-27,2021-01-28,13971.3,13950,10000,980.7,855.00,125.7,1983.0,1841.80,141.2,2978.95,2867.65,111.3,<NA>,NaN,NaN


In [3]:
train_dates[["trade_day_before_expiry", "atm_strike", "nifty_at_0320", "call_at_0320_600", "call_at_0918_600", "call_pnl_0918_600"]]

,trade_day_before_expiry,atm_strike,nifty_at_0320,call_at_0320_600,call_at_0918_600,call_pnl_0918_600
trade_date,,,,,,
2021-01-06,2021-01-06,14150,14147.6,596.15,670.20,-74.05
2021-01-13,2021-01-13,14550,14566.6,625.9,599.40,26.5
2021-01-20,2021-01-20,14650,14634.0,575.85,658.50,-82.65
2021-01-27,2021-01-27,13950,13971.3,<NA>,NaN,NaN
2021-02-10,2021-02-10,15100,15120.8,629.15,582.20,46.95
2021-02-17,2021-02-17,15200,15208.2,605.0,614.95,-9.95
2021-04-07,2021-04-07,14800,14821.15,628.0,717.00,-89.0
2021-05-19,2021-05-19,15000,15019.85,637.0,650.00,-13.0
2021-05-26,2021-05-26,15300,15307.95,610.9,605.00,5.9


In [7]:
train_dates[["trade_day_before_expiry", "atm_strike", "nifty_at_0320", "put_at_0320_250", "put_at_0918_250", "put_pnl_0918_250"]]
print(train_dates[train_dates.put_pnl_0918_250 > 0].shape)
print(train_dates[train_dates.put_pnl_0918_250 <= 0].shape)
train_dates.shape

# train_dates

(39, 81)
(29, 81)


(68, 81)

In [5]:
train_dates[["trade_day_before_expiry", "atm_strike", "nifty_at_0320", "put_at_0320_250", "put_at_0918_250", "put_pnl_0918_250"]]
# train_dates.to_csv("time_decay_results.csv")

,trade_day_before_expiry,atm_strike,nifty_at_0320,put_at_0320_250,put_at_0918_250,put_pnl_0918_250
trade_date,,,,,,
2021-02-03,2021-02-03,14800,14787.50,271.65,281.80,-10.15
2021-03-03,2021-03-03,15250,15259.00,244.65,457.80,-213.15
2021-03-09,2021-03-09,15100,15120.05,241.50,153.05,88.45
2021-03-17,2021-03-17,14700,14714.35,227.35,118.30,109.05
2021-03-24,2021-03-24,14550,14544.80,254.35,320.00,-65.65
2021-03-31,2021-03-31,14700,14686.90,256.85,151.80,105.05
2021-04-28,2021-04-28,14850,14859.20,255.80,102.10,153.70
2021-05-05,2021-05-05,14600,14620.85,220.25,194.95,25.30
2021-05-11,2021-05-11,14850,14850.40,254.20,296.00,-41.80


In [14]:
train_dates.loc[train_dates.pnl_0915 < 0, ["expiry", "atm_strike", "call_at_0320", "call_at_0915", "put_at_0320", "put_at_0915", "call_pnl_0915", "put_pnl_0915", "pnl_0918"]]

,expiry,atm_strike,call_at_0320,call_at_0915,put_at_0320,put_at_0915,call_pnl_0915,put_pnl_0915,pnl_0918
trade_date,,,,,,,,,
2021-01-27,2021-01-28,13950,155.40,63.20,228.05,333.75,92.20,-105.70,-36.45
2021-06-16,2021-06-17,15750,130.40,68.40,247.15,314.35,62.00,-67.20,-6.20
2021-07-07,2021-07-08,15900,90.15,80.15,271.45,292.05,10.00,-20.60,-2.25
2021-10-27,2021-10-28,18200,144.30,99.20,236.50,294.30,45.10,-57.80,-14.40
2021-12-01,2021-12-02,17200,126.40,101.60,263.15,291.60,24.80,-28.45,2.40
2021-12-29,2021-12-30,17200,133.95,93.20,240.05,281.70,40.75,-41.65,4.20
2022-03-09,2022-03-10,16350,183.45,390.70,267.60,64.80,-207.25,202.80,-10.45
2022-05-18,2022-05-19,16250,129.10,13.25,271.05,527.80,115.85,-256.75,-181.80
2022-06-08,2022-06-09,16350,129.40,63.20,257.50,333.95,66.20,-76.45,-7.65


In [5]:
train_dates = pd.read_csv("time_decay_results.csv")

,trade_date,previous_trading_day,expiry,trade_day_before_expiry,nifty_at_0320,atm_strike,cur_put_atm_strike,put_at_0320_-500,put_at_0918_-500,put_pnl_0918_-500,...,put_pnl_0918_550,put_at_0320_600,put_at_0918_600,put_pnl_0918_600,put_at_0320_650,put_at_0918_650,put_pnl_0918_650,put_at_0320_700,put_at_0918_700,put_pnl_0918_700
0,2021-02-03,2021-02-02,2021-02-04,2021-02-03,14787.50,14800,15500,5.35,1.85,3.50,...,-32.95,615.85,654.40,-38.55,670.00,704.85,-34.85,717.50,735.45,-17.95
1,2021-03-03,2021-03-02,2021-03-04,2021-03-03,15259.00,15250,15950,5.65,10.05,-4.40,...,-194.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-03-09,2021-03-08,2021-03-10,2021-03-09,15120.05,15100,15800,2.90,0.95,1.95,...,79.95,594.45,501.90,92.55,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-03-17,2021-03-16,2021-03-18,2021-03-17,14714.35,14700,15400,1.55,0.35,1.20,...,116.35,565.75,448.70,117.05,624.15,499.75,124.40,662.95,547.70,115.25
4,2021-03-24,2021-03-23,2021-03-25,2021-03-24,14544.80,14550,15250,3.00,2.35,0.65,...,-68.70,599.15,682.00,-82.85,644.35,718.30,-73.95,694.70,780.20,-85.50
5,2021-03-31,2021-03-30,2021-04-01,2021-03-31,14686.90,14700,15400,3.25,1.05,2.20,...,63.65,602.95,494.00,108.95,652.85,585.15,67.70,701.00,613.05,87.95
6,2021-04-28,2021-04-27,2021-04-29,2021-04-28,14859.20,14850,15550,6.00,1.70,4.30,...,164.10,NaN,NaN,NaN,649.00,488.60,160.40,NaN,NaN,NaN
7,2021-05-05,2021-05-04,2021-05-06,2021-05-05,14620.85,14600,15300,2.25,0.70,1.55,...,17.90,566.00,550.15,15.85,617.30,561.95,55.35,665.00,641.40,23.60
8,2021-05-11,2021-05-10,2021-05-12,2021-05-11,14850.40,14850,15550,1.90,0.75,1.15,...,-64.45,NaN,NaN,NaN,651.00,717.20,-66.20,NaN,NaN,NaN
9,2021-06-02,2021-06-01,2021-06-03,2021-06-02,15587.45,15600,16300,1.80,0.75,1.05,...,NaN,NaN,NaN,NaN,661.35,599.15,62.20,NaN,NaN,NaN


In [10]:
train_dates["rolling_sum_7"] = train_dates["put_pnl_0918_250"].rolling(window=1).sum()
train_dates["rolling_sum_10"] = train_dates["put_pnl_0918_250"].rolling(window=10).sum()

In [11]:
train_dates[["trade_day_before_expiry", "atm_strike", "nifty_at_0320", "put_at_0320_250", "put_at_0918_250", "put_pnl_0918_250", "rolling_sum_7", "rolling_sum_10"]]

,trade_day_before_expiry,atm_strike,nifty_at_0320,put_at_0320_250,put_at_0918_250,put_pnl_0918_250,rolling_sum_7,rolling_sum_10
0,2021-02-03,14800,14787.50,271.65,281.80,-10.15,NaN,NaN
1,2021-03-03,15250,15259.00,244.65,457.80,-213.15,NaN,NaN
2,2021-03-09,15100,15120.05,241.50,153.05,88.45,NaN,NaN
3,2021-03-17,14700,14714.35,227.35,118.30,109.05,NaN,NaN
4,2021-03-24,14550,14544.80,254.35,320.00,-65.65,NaN,NaN
5,2021-03-31,14700,14686.90,256.85,151.80,105.05,NaN,NaN
6,2021-04-28,14850,14859.20,255.80,102.10,153.70,167.30,NaN
7,2021-05-05,14600,14620.85,220.25,194.95,25.30,202.75,NaN
8,2021-05-11,14850,14850.40,254.20,296.00,-41.80,374.10,NaN
9,2021-06-02,15600,15587.45,261.00,206.40,54.60,340.25,205.40
